In [76]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[autoreload of lynx.table failed: Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniconda/base/envs/indb-fm/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 273, in check
    superreload(m, reload, self.old_objects)
  File "/opt/homebrew/Caskroom/miniconda/base/envs/indb-fm/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 471, in superreload
    module = reload(module)
  File "/opt/homebrew/Caskroom/miniconda/base/envs/indb-fm/lib/python3.10/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 619, in _exec
  File "<frozen importlib._bootstrap_external>", line 883, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/Users/austin-pan/Documents/lynx/lynxx/lynx/table/__init__.py", line 1, in <module>
    from lynx.table.table import (
ImportError: cannot import name 'model_interactions' from 'lynx.table.table'

In [77]:
import pandas as pd
import numpy as np

import lynx as lx
from lynx.datasets import movielens

dataset_path = "~/Downloads/lynx/datasets/movielens/ml-1m"

In [78]:
if False:
    movielens.download(destination="~/Downloads/lynx/datasets/movielens")

In [79]:
users = movielens.load_users(
    dataset_path,
    usecols=["user_id", "gender", "age", "occupation"],
    nrows=1000
)
users.head()

,user_id,gender,age,occupation
0,1,F,1,10
1,2,M,56,16
2,3,M,25,15
3,4,M,45,7
4,5,M,25,20


In [80]:
users_table = lx.Table(users, "users")

users_table = (
    users_table
    .onehot("age")
    .onehot("gender")
    .onehot("occupation")
)
users_table.to_dataframe().head()

,user_id,0,1,2,3,4,5,6,0,1,...,11,12,13,14,15,16,17,18,19,20
0,1,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,1,0,0,0,0
2,3,0,0,1,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
3,4,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,5,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1


In [81]:
movies = movielens.load_movies(
    dataset_path,
    usecols=["movie_id", "genres"],
    nrows=1000
)
movies["genres"] = movies["genres"].str.split("|")
movies.head()

,movie_id,genres
0,1,"[Animation, Children's, Comedy]"
1,2,"[Adventure, Children's, Fantasy]"
2,3,"[Comedy, Romance]"
3,4,"[Comedy, Drama]"
4,5,[Comedy]


In [82]:
movies_table = lx.Table(movies, "movies")

movies_table = movies_table.explode("genres")
movies_table.to_dataframe().head()

,movie_id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
3,4,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [83]:
ratings = movielens.load_ratings(
    dataset_path,
    usecols=["user_id", "movie_id", "rating"],
    nrows=10000
)
ratings.head()

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [84]:
ratings_table = lx.Table(ratings, "ratings")
ratings_table.to_dataframe().head()

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [85]:
merged_table = (
    ratings_table
    .merge(movies_table, left_on="movie_id", right_on="movie_id")
    .merge(users_table, left_on="user_id", right_on="user_id")
    .model_interactions("user_id", "movie_id")
    .onehot("movie_id")
    .onehot("user_id")
)

print(merged_table.shape)
print(merged_table.block_shapes)

(2544, 1227)
{'ratings': (10000, 1), 'genres_exploded': (154, 18), 'age_onehot': (7, 7), 'gender_onehot': (2, 2), 'occupation_onehot': (21, 21), 'user_id_movie_id_interactions': (70, 554), 'movie_id_onehot': (554, 554), 'user_id_onehot': (70, 70)}


# Experiments

In [86]:
from sklearn.model_selection import train_test_split

def rmse(predictions, targets) -> float:
    return np.sqrt(np.mean((predictions - targets) ** 2))

seed = 0

y = merged_table.pop("rating")
X_train, X_test, y_train, y_test = train_test_split(
    merged_table, y,
    train_size=0.8,
    test_size=0.2,
    random_state=seed
)

## Dense Structure (X)


In [87]:
from lynx.libfm import mcmc

mcmc_r = mcmc.FMRegression(seed=seed)
pred = mcmc_r.fit_predict(X_train, y_train, X_test, verbose=True)
mcmc_r.flush()

rmse(pred, y_test)

Converting /tmp/libfm_mcmc_r_171317469581806/train
Transposing /tmp/libfm_mcmc_r_171317469581806/train
Converting /tmp/libfm_mcmc_r_171317469581806/test
Transposing /tmp/libfm_mcmc_r_171317469581806/test
/Users/austin-pan/Documents/libfm/bin/libFM -method mcmc -task r -train train -test test -dim 1,1,8 -iter 100 -init_stdev 0.1 -out /tmp/libfm_mcmc_r_171317469581806/predictions.txt -seed 0
----------------------------------------------------------------------------
libFM
  Version: 1.4.4
  Author:  Steffen Rendle, srendle@libfm.org
  WWW:     http://www.libfm.org/
This program comes with ABSOLUTELY NO WARRANTY; for details see license.txt.
This is free software, and you are welcome to redistribute it under certain
conditions; for details see license.txt.
----------------------------------------------------------------------------
Loading train...
has x = 0
has xt = 1
data transpose... num entries in cache=80812	num rows in cache=1226
num_cases=2035	num_values=80812	num_features=1226	mi

0.9186495703174311

In [88]:
from lynx.libfm import als

sgda_r = als.FMRegression(regularizations=(0,0,10), seed=seed)
sgda_r.fit(X_train, y_train, verbose=True)
pred = sgda_r.predict(X_test, verbose=True)
sgda_r.flush()

rmse(pred, y_test)

Converting /tmp/libfm_als_r_1713174696274621/train
Transposing /tmp/libfm_als_r_1713174696274621/train
Converting /tmp/libfm_als_r_1713174696274621/test
Transposing /tmp/libfm_als_r_1713174696274621/test
/Users/austin-pan/Documents/libfm/bin/libFM -method als -task r -train train -test test -dim 1,1,8 -iter 100 -init_stdev 0.1 -regular 0,0,10 -save_model als_model -seed 0
----------------------------------------------------------------------------
libFM
  Version: 1.4.4
  Author:  Steffen Rendle, srendle@libfm.org
  WWW:     http://www.libfm.org/
This program comes with ABSOLUTELY NO WARRANTY; for details see license.txt.
This is free software, and you are welcome to redistribute it under certain
conditions; for details see license.txt.
----------------------------------------------------------------------------
Loading train...
has x = 0
has xt = 1
data transpose... num entries in cache=80812	num rows in cache=1226
num_cases=2035	num_values=80812	num_features=1226	min_target=1	max_tar

1.0414252724757918

In [89]:
from lynx.libfm import sgd

sgda_r = sgd.FMRegression(learn_rate=0.001, seed=seed)
sgda_r.fit(X_train, y_train, verbose=True)
pred = sgda_r.predict(X_test, verbose=True)
sgda_r.flush()

rmse(pred, y_test)

Converting /tmp/libfm_sgd_r_1713174696698165/train
Transposing /tmp/libfm_sgd_r_1713174696698165/train
Converting /tmp/libfm_sgd_r_1713174696698165/test
Transposing /tmp/libfm_sgd_r_1713174696698165/test
/Users/austin-pan/Documents/libfm/bin/libFM -method sgd -task r -train train -test test -dim 1,1,8 -iter 100 -init_stdev 0.1 -learn_rate 0.001 -save_model sgd_model -seed 0
----------------------------------------------------------------------------
libFM
  Version: 1.4.4
  Author:  Steffen Rendle, srendle@libfm.org
  WWW:     http://www.libfm.org/
This program comes with ABSOLUTELY NO WARRANTY; for details see license.txt.
This is free software, and you are welcome to redistribute it under certain
conditions; for details see license.txt.
----------------------------------------------------------------------------
Loading train...
has x = 1
has xt = 0
data... num entries in cache=80812	num rows in cache=2035
num_cases=2035	num_values=80812	num_features=1226	min_target=1	max_target=5
Lo

1.0016610036249327

In [90]:
from lynx.libfm import sgda

X_val, X_sgda_test, y_val, y_sgda_test = train_test_split(
    X_test, y_test,
    train_size=0.5,
    test_size=0.5,
    random_state=seed
)

print(X_train.shape)
print(X_val.shape)
print(X_sgda_test.shape)

sgda_r = sgda.FMRegression(X_val, y_val, learn_rate=0.001, seed=seed)
sgda_r.fit(X_train, y_train, verbose=True)
pred = sgda_r.predict(X_sgda_test, verbose=True)
sgda_r.flush()

rmse(pred, y_sgda_test)

(2035, 1226)
(254, 1226)
(255, 1226)
Converting /tmp/libfm_sgda_r_1713174697292491/train
Transposing /tmp/libfm_sgda_r_1713174697292491/train
Converting /tmp/libfm_sgda_r_1713174697292491/test
Transposing /tmp/libfm_sgda_r_1713174697292491/test
/Users/austin-pan/Documents/libfm/bin/libFM -method sgda -task r -train train -test test -dim 1,1,8 -iter 100 -init_stdev 0.1 -learn_rate 0.001 -save_model sgda_model -seed 0 -validation validation
----------------------------------------------------------------------------
libFM
  Version: 1.4.4
  Author:  Steffen Rendle, srendle@libfm.org
  WWW:     http://www.libfm.org/
This program comes with ABSOLUTELY NO WARRANTY; for details see license.txt.
This is free software, and you are welcome to redistribute it under certain
conditions; for details see license.txt.
----------------------------------------------------------------------------
Loading train...
has x = 1
has xt = 0
data... num entries in cache=80812	num rows in cache=2035
num_cases=20

0.9662177889723759

## Block Structure (BS)

In [98]:
from lynx.libfm.bs import mcmc as mcmc_bs

mcmc_r = mcmc_bs.FMRegression(seed=seed)
pred = mcmc_r.fit_predict(X_train, y_train, X_test, verbose=True)
mcmc_r.flush()

rmse(pred, y_test)

Converting /tmp/libfm_mcmc_r_1713175283920775/genres_exploded
Transposing /tmp/libfm_mcmc_r_1713175283920775/genres_exploded
Converting /tmp/libfm_mcmc_r_1713175283920775/age_onehot
Transposing /tmp/libfm_mcmc_r_1713175283920775/age_onehot
Converting /tmp/libfm_mcmc_r_1713175283920775/gender_onehot
Transposing /tmp/libfm_mcmc_r_1713175283920775/gender_onehot
Converting /tmp/libfm_mcmc_r_1713175283920775/occupation_onehot
Transposing /tmp/libfm_mcmc_r_1713175283920775/occupation_onehot
Converting /tmp/libfm_mcmc_r_1713175283920775/user_id_movie_id_interactions
Transposing /tmp/libfm_mcmc_r_1713175283920775/user_id_movie_id_interactions
Converting /tmp/libfm_mcmc_r_1713175283920775/movie_id_onehot
Transposing /tmp/libfm_mcmc_r_1713175283920775/movie_id_onehot
Converting /tmp/libfm_mcmc_r_1713175283920775/user_id_onehot
Transposing /tmp/libfm_mcmc_r_1713175283920775/user_id_onehot
/Users/austin-pan/Documents/libfm/bin/libFM -method mcmc -task r -train train -test test -dim 1,1,8 -iter 100

0.9327757319303459

In [92]:
from lynx.libfm.bs import als as als_bs

sgda_r = als_bs.FMRegression(regularizations=(0,0,10), seed=seed)
sgda_r.fit(X_train, y_train, verbose=True)
pred = sgda_r.predict(X_test, verbose=True)
sgda_r.flush()

rmse(pred, y_test)

Converting /tmp/libfm_als_r_17131747452750452/genres_exploded
Transposing /tmp/libfm_als_r_17131747452750452/genres_exploded
Converting /tmp/libfm_als_r_17131747452750452/age_onehot
Transposing /tmp/libfm_als_r_17131747452750452/age_onehot
Converting /tmp/libfm_als_r_17131747452750452/gender_onehot
Transposing /tmp/libfm_als_r_17131747452750452/gender_onehot
Converting /tmp/libfm_als_r_17131747452750452/occupation_onehot
Transposing /tmp/libfm_als_r_17131747452750452/occupation_onehot
Converting /tmp/libfm_als_r_17131747452750452/user_id_movie_id_interactions
Transposing /tmp/libfm_als_r_17131747452750452/user_id_movie_id_interactions
Converting /tmp/libfm_als_r_17131747452750452/movie_id_onehot
Transposing /tmp/libfm_als_r_17131747452750452/movie_id_onehot
Converting /tmp/libfm_als_r_17131747452750452/user_id_onehot
Transposing /tmp/libfm_als_r_17131747452750452/user_id_onehot
/Users/austin-pan/Documents/libfm/bin/libFM -method als -task r -train train -test test -dim 1,1,8 -iter 100 

1.0414252790284144